In [1]:
import os
import lightgbm as lgb
import pandas as pd
import warnings
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import optuna
import random
from datetime import datetime
import pytz

# importo los .py
import sys
sys.path.append("auxiliares")
import config
import metricas
import lightgbm_aux
import extras

pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_columns', None)

# Ignore all warnings
warnings.filterwarnings("ignore")

In [2]:
####################################################
############# Setear segun cada maquina ############
#os.chdir("C:/Users/herna/labo3_empresa3_repo/")
#os.chdir("C:/diego_tools/labo3/")
os.chdir("/home/dcastro_contacto/buckets/b1/")
####################################################

In [3]:
#Manual
prefijo_arch_entrada = "datasets/emp3_sellout_fe_cluster_manual_"
num_clusters=94
carpeta_exp_prefijo = "exp/avanzada_1/"

#Time Warping
#prefijo_arch_entrada = "datasets/emp3_sellout_base_cluster_warp_"
#prefijo_arch_salida = "datasets/emp3_sellout_fe_cluster_warp_"
#num_clusters=94
#carpeta_exp_prefijo = "exp/avanzada_2/"

In [4]:
def lgbm_error_rate(preds, train_data):
    labels = train_data.get_label()
    ret_val = metricas.error_rate(labels,preds)
    return 'ER', ret_val, False

In [5]:
def objective(trial):
    print("****************", config.OBJECTIVE_PARAM, "****************")
    
    lgb_params = {
        'objective': config.OBJECTIVE_PARAM,
        'first_metric_only': True,
        'boost_from_average': True,
        'max_depth':-1,
        'lambda_l1': trial.suggest_uniform('lambda_l1', 0.0,config.L1_UPPER_PARAM),
        'lambda_l2': trial.suggest_uniform('lambda_l2', 0.0,config.L2_UPPER_PARAM),
        'min_gain_to_split':0.0,
        'force_row_wise':True,
        'feature_pre_filter':False,
        'metric': "None",
        'max_bin': config.MAX_BIN_PARAM,
        'learning_rate': trial.suggest_uniform('learning_rate', config.LEARNING_RATE_LOWER_PARAM,config.LEARNING_RATE_UPPER_PARAM ),
        'num_leaves' : trial.suggest_int('num_leaves', config.NUM_LEAVES_LOWER_PARAM, config.NUM_LEAVES_UPPER_PARAM),
        'feature_fraction': trial.suggest_uniform('feature_fraction', config.FEATURE_FRACTION_LOWER_PARAM, config.FEATURE_FRACTION_UPPER_PARAM),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', config.MIN_DATA_IN_LEAF_LOWER_PARAM, config.MIN_DATA_IN_LEAF_UPPER_PARAM),
        #'bagging_freq':trial.suggest_int('bagging_freq', 1, 10),
        #'bagging_fraction': trial.suggest_categorical('bagging_fraction',[0.7,0.75,0.8,0.85,0.9,0.95]),
        #'extra_trees':trial.suggest_categorical('extra_trees',[True,False]),
        'verbose':-100,
        'num_threads':-1
    }
    
    lgb_params_num_boost_round = 10000  #un numero muy grande, lo limita early_stopping_rounds
    lgb_params_early_stopping_rounds = int(50 + 5/lgb_params['learning_rate']) #  #el parametro discolo, que depende de otro
    
    lgb_train = lgb.Dataset(data=X_train, label=y_train, feature_name=cols_entren)
    lgb_validate = lgb.Dataset(data=X_validate, label=y_validate, reference=lgb_train, feature_name=cols_entren)
    
    #Train using selected parameters
    model = lgb.train(lgb_params, lgb_train,
                  valid_sets=[lgb_validate],
                  num_boost_round=lgb_params_num_boost_round,
                  early_stopping_rounds=lgb_params_early_stopping_rounds,
                  feval=lgbm_error_rate,
                  verbose_eval=100)
    
    y_pred_validate =model.predict(X_validate) #Create predictions on test set
    er_validate = metricas.error_rate(y_validate,y_pred_validate)
    
    GLOBAL_PARAMS_ITER.append(lgb_params)
    GLOBAL_BEST_IT_ITER.append(model.best_iteration)
    GLOBAL_ER_VALIDATE_ITER.append(er_validate)
    GLOBAL_MODEL_ITER.append(model)
    
    return er_validate

In [6]:
##############################################

In [7]:
current_datetime = datetime.now().astimezone(pytz.timezone('America/Argentina/Buenos_Aires'))
exp_numero = current_datetime.strftime("%Y%m%d %H%M%S").replace(" ", "_")
exp_numero

'20231212_163228'

In [8]:
# Guardo Info del experimento
carpeta_exp = carpeta_exp_prefijo + exp_numero + "/"
if not os.path.exists(carpeta_exp):
    os.makedirs(carpeta_exp)
    
# Parametros
parametros_nombres = ["PERIODO_INICIO_PARAM","TRAIN_PERIODO_LIMITE_PARAM","VALIDATE_PERIODO_PARAM","USAR_HOLDOUT_PARAM",
                      "TRAIN_ALL_PERIODO_LIMITE_PARAM","HOLDOUT_PERIODO_PARAM",
                      "MODELO_FINAL_PERIODO_LIMITE_PARAM","FUTURE_PERIODO_PARAM",
                      "ARCH_PRODUCTOS_PREDICCION_ENTRENAMIENTO","ARCH_PRODUCTOS_PREDICCION_FUTURE",
                      "NUM_LAGS_PARAM","FAMILIA_FEATURES_TEMP_PARAM",
                      "CANT_ITERACIONES_OPT_BAY_PARAM","CANT_SEMILLAS"]

parametros_valores = [config.PERIODO_INICIO_PARAM,config.TRAIN_PERIODO_LIMITE_PARAM,config.VALIDATE_PERIODO_PARAM,
                      config.USAR_HOLDOUT_PARAM,config.TRAIN_ALL_PERIODO_LIMITE_PARAM,config.HOLDOUT_PERIODO_PARAM,
                      config.MODELO_FINAL_PERIODO_LIMITE_PARAM,config.FUTURE_PERIODO_PARAM,
                      config.ARCH_PRODUCTOS_PREDICCION_ENTRENAMIENTO,config.ARCH_PRODUCTOS_PREDICCION_FUTURE,
                      config.NUM_LAGS_PARAM,config.FAMILIA_FEATURES_TEMP_PARAM,
                      config.CANT_ITERACIONES_OPT_BAY_PARAM, config.CANT_SEMILLAS]

parametros = pd.DataFrame(data={"nombre":parametros_nombres,"valor":parametros_valores})
parametros.to_csv(carpeta_exp + "parametros.csv",index=False)

In [ ]:
# Entrenamiento por cluster
er_validate_iter = [None] * num_clusters
er_holdout_semillerio_iter = [None] * num_clusters
params_iter = [None] * num_clusters
best_iter_iter = [None] * num_clusters
pred_iter = [None] * num_clusters
prod_iter = [None] * num_clusters

for i in range(0,num_clusters):
    print("=================== Cluster", i, "===================")
    
    df_sellout = pd.read_csv(prefijo_arch_entrada + str(i) + ".csv")
    
    ### Categoricas
    df_sellout = extras.convertir_categoricas_prod_cust(df_sellout)

    ### Remover columnas
    df_sellout = df_sellout.drop(columns=["periodo_fecha","brand"])
    
    ### Split de datasets
    df_train = df_sellout[(df_sellout.periodo <= config.TRAIN_PERIODO_LIMITE_PARAM) & (df_sellout.periodo >= config.PERIODO_INICIO_PARAM)]
    df_validate = df_sellout[(df_sellout.periodo == config.VALIDATE_PERIODO_PARAM)]
    
    print("Periodos entrenar:",df_train.periodo.unique())
    print("Periodos validar:",df_validate.periodo.unique())
    
    df_validate = df_validate.sort_values(by="product_id",ascending=True)
    print("product_id a validar:", len(df_validate))

    ### Armado variables entrenamiento
    cols_remover_entren = ["tn_mas_2","periodo"]
    
    # Train - Validate
    X_train = df_train.drop(columns=cols_remover_entren, axis=1)
    X_validate = df_validate.drop(columns=cols_remover_entren, axis=1)
    
    y_train = df_train.tn_mas_2
    y_validate = df_validate.tn_mas_2
    
    print("X_train:", X_train.shape)
    print("y_train:", y_train.shape)
    
    print("\nX_validate:", X_validate.shape)
    print("y_validate:", y_validate.shape)
    
    cols_entren = X_train.columns.tolist()

    ### Optimizacion Bayesiana
    print("Optimizacion Bayesiana")
    GLOBAL_PARAMS_ITER = []
    GLOBAL_BEST_IT_ITER = []
    GLOBAL_ER_VALIDATE_ITER = []
    GLOBAL_MODEL_ITER = []
    
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=config.CANT_ITERACIONES_OPT_BAY_PARAM)
    print('# Experimentos:', len(study.trials))
    print('Mejor experimento:', study.best_trial.params)

    best_model_index = study.best_trial.number
    best_model = GLOBAL_MODEL_ITER[best_model_index]

    er_validate_iter[i]=study.best_trial.value
    params_iter[i] = GLOBAL_PARAMS_ITER[best_model_index]
    best_iter_iter[i] = GLOBAL_BEST_IT_ITER[best_model_index] 

Cluster 0


[I 2023-12-12 19:32:49,844] A new study created in memory with name: no-name-b60594c4-85fb-4ea8-9420-29a0c4257a85


Periodos entrenar: [201701 201702 201703 201704 201705 201706 201707 201708 201709 201710
 201711 201712 201801 201802 201803 201804 201805 201806 201807 201808
 201809 201810]
Periodos validar: [201812]
product_id a validar: 27900
X_train: (469490, 120)
y_train: (469490,)

X_validate: (27900, 120)
y_validate: (27900,)
Optimizacion Bayesiana
**************** regression ****************
Training until validation scores don't improve for 70 rounds


[I 2023-12-12 19:32:55,288] Trial 0 finished with value: 114.35 and parameters: {'lambda_l1': 241.3244922856699, 'lambda_l2': 640.0817297652003, 'learning_rate': 0.24202692382438273, 'num_leaves': 786, 'feature_fraction': 0.21141011116231834, 'min_data_in_leaf': 5963}. Best is trial 0 with value: 114.35.


[100]	valid_0's ER: 121.2
Early stopping, best iteration is:
[33]	valid_0's ER: 114.35
Evaluated only: ER
**************** regression ****************
Training until validation scores don't improve for 69 rounds
[100]	valid_0's ER: 126.47


...

**************** regression ****************
Training until validation scores don't improve for 74 rounds
[100]	valid_0's ER: 102.39


[I 2023-12-12 19:37:10,056] Trial 1 finished with value: 99.46 and parameters: {'lambda_l1': 782.817993009741, 'lambda_l2': 79.10227073280984, 'learning_rate': 0.2048895933968013, 'num_leaves': 20, 'feature_fraction': 0.24598106239806025, 'min_data_in_leaf': 2268}. Best is trial 1 with value: 99.46.


Early stopping, best iteration is:
[56]	valid_0's ER: 99.46
Evaluated only: ER
**************** regression ****************
Training until validation scores don't improve for 160 rounds
[100]	valid_0's ER: 115.4
[200]	valid_0's ER: 114.31
[300]	valid_0's ER: 113.06
[400]	valid_0's ER: 112.15
[500]	valid_0's ER: 111.32
[600]	valid_0's ER: 110.73
[700]	valid_0's ER: 110.1
[800]	valid_0's ER: 109.59
[900]	valid_0's ER: 109.03
[1000]	valid_0's ER: 108.88
[1100]	valid_0's ER: 108.56
[1200]	valid_0's ER: 108.33
[1300]	valid_0's ER: 107.98
[1400]	valid_0's ER: 107.48
[1500]	valid_0's ER: 107.21
[1600]	valid_0's ER: 106.95
[1700]	valid_0's ER: 106.56
[1800]	valid_0's ER: 106.77
Early stopping, best iteration is:
[1700]	valid_0's ER: 106.56
Evaluated only: ER


[I 2023-12-12 19:40:16,652] Trial 2 finished with value: 106.56 and parameters: {'lambda_l1': 684.1999270988266, 'lambda_l2': 89.4931414086243, 'learning_rate': 0.0453193639995204, 'num_leaves': 748, 'feature_fraction': 0.9230507924159748, 'min_data_in_leaf': 3825}. Best is trial 1 with value: 99.46.


**************** regression ****************
Training until validation scores don't improve for 104 rounds
[100]	valid_0's ER: 114.13
[200]	valid_0's ER: 109.27
[300]	valid_0's ER: 106.15
[400]	valid_0's ER: 102.3
[500]	valid_0's ER: 100.68
[600]	valid_0's ER: 99
[700]	valid_0's ER: 97.6
[800]	valid_0's ER: 96.34
[900]	valid_0's ER: 95.38
[1000]	valid_0's ER: 93.35
[1100]	valid_0's ER: 93.13
[1200]	valid_0's ER: 92.48
[1300]	valid_0's ER: 91.87
[1400]	valid_0's ER: 91.18
[1500]	valid_0's ER: 90.53
[1600]	valid_0's ER: 89.93


In [ ]:
df_resultados_opt_bay = pd.DataFrame(data={"cluster":range(0,num_clusters),"error_validate":er_validate_iter,
                                   "lgbm_params":params_iter,"best_iteration":best_iter_iter})

df_resultados_opt_bay.to_excel(carpeta_exp +  "opt_bay.xlsx",index=False)

df_resultados_opt_bay

In [ ]:
print("FINALIZADO")